# Netflix Stock Analysis with Trading Signal
This notebook downloads 5 years of Netflix (NFLX) stock data from Yahoo Finance, visualizes the data using candlestick charts, and implements a profitable trading signal based on candlestick patterns and technical indicators.

Notebook created by: Alan Nadelsticher

Copiloted by: Claude Sonnet 4.5

In [13]:
# Install required packages if needed
# !pip install yfinance plotly pandas

In [14]:
# Import required libraries
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np

In [15]:
# Define the stock ticker and date range (5 years)
ticker = 'NFLX'
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)  # 5 years of data

print(f"Downloading {ticker} data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")
print(f"Total period: 5 years")

Total period: 5 years


In [16]:
# Download Netflix stock data from Yahoo Finance
nflx_data = yf.download(ticker, start=start_date, end=end_date)

# Flatten multi-level columns if they exist
if isinstance(nflx_data.columns, pd.MultiIndex):
    nflx_data.columns = nflx_data.columns.get_level_values(0)

# Display first few rows
print(f"\nData shape: {nflx_data.shape}")
print(f"Date range: {nflx_data.index[0]} to {nflx_data.index[-1]}")
nflx_data.head()

/var/folders/0b/xzyxlt490hdb6dvwbdsw46x40000gn/T/ipykernel_45061/3211705483.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


Data shape: (1255, 5)
Date range: 2020-10-23 00:00:00 to 2025-10-22 00:00:00


Price,Close,High,Low,Open,Volume
Date,,,,,
2020-10-23,488.279999,490.059998,481.350006,488.109985,4927900
2020-10-26,488.239990,496.820007,478.899994,487.029999,6186100
2020-10-27,488.929993,490.489990,482.929993,490.010010,3627200
2020-10-28,486.239990,494.000000,483.279999,486.359985,5992700
2020-10-29,504.209991,513.900024,479.339996,488.500000,11120700


In [17]:
# Display summary statistics
print("Summary Statistics (5 Years):")
nflx_data[['Open', 'High', 'Low', 'Close', 'Volume']].describe()

Summary Statistics (5 Years):


Price,Open,High,Low,Close,Volume
count,1255.000000,1255.000000,1255.000000,1255.000000,1.255000e+03
mean,579.325482,587.140692,571.524557,579.555561,5.758823e+06
std,283.424697,286.036751,280.605160,283.672595,6.189635e+06
min,163.960007,172.059998,162.710007,166.369995,1.144000e+06
25%,377.294998,382.619995,368.119995,376.034988,2.941000e+06
50%,518.500000,525.409973,512.969971,519.200012,4.094500e+06
75%,674.945007,683.420013,669.025024,675.625000,6.417450e+06
max,1338.229980,1341.150024,1321.209961,1339.130005,1.333875e+08


In [18]:
# Create candlestick chart for full 5-year period
fig = go.Figure(data=[go.Candlestick(
    x=nflx_data.index,
    open=nflx_data['Open'],
    high=nflx_data['High'],
    low=nflx_data['Low'],
    close=nflx_data['Close'],
    name='NFLX'
)])

# Update layout for better visualization with range slider
fig.update_layout(
    title=f'Netflix (NFLX) Stock Price - 5 Year History',
    yaxis_title='Stock Price (USD)',
    xaxis_title='Date',
    template='plotly_white',
    xaxis_rangeslider_visible=True,  # Enable range slider for zooming
    height=600,
    width=1200
)

fig.show()

In [19]:
# Calculate Technical Indicators for Trading Signal
# We'll use 50-day moving average (50 DMA) and 200-day moving average (200 DMA)
nflx_data['SMA_50'] = nflx_data['Close'].rolling(window=50).mean()
nflx_data['SMA_200'] = nflx_data['Close'].rolling(window=200).mean()

# Calculate body and wick sizes for candlestick pattern detection
nflx_data['Body'] = abs(nflx_data['Close'] - nflx_data['Open'])
nflx_data['Upper_Wick'] = nflx_data['High'] - nflx_data[['Open', 'Close']].max(axis=1)
nflx_data['Lower_Wick'] = nflx_data[['Open', 'Close']].min(axis=1) - nflx_data['Low']
nflx_data['Range'] = nflx_data['High'] - nflx_data['Low']

# Identify bullish patterns (Hammer and Bullish Engulfing)
# Hammer: small body at top, long lower wick (at least 2x body), bullish candle
nflx_data['Is_Green'] = nflx_data['Close'] > nflx_data['Open']
nflx_data['Is_Hammer'] = (
    (nflx_data['Lower_Wick'] > 2 * nflx_data['Body']) &  # Long lower wick
    (nflx_data['Upper_Wick'] < 0.5 * nflx_data['Body']) &  # Small upper wick
    (nflx_data['Body'] > 0) &  # Has a body
    (nflx_data['Is_Green'])  # Green candle
)

# Display first detection
print("Technical indicators calculated:")
print(f"- 50-day Simple Moving Average (SMA_50)")
print(f"- 200-day Simple Moving Average (SMA_200)")
print(f"- Hammer candlestick pattern detection")
print(f"\nHammers detected: {nflx_data['Is_Hammer'].sum()}")
nflx_data.tail(10)

Technical indicators calculated:
- 50-day Simple Moving Average (SMA_50)
- 200-day Simple Moving Average (SMA_200)
- Hammer candlestick pattern detection

Hammers detected: 17


Price,Close,High,Low,Open,Volume,SMA_50,SMA_200,Body,Upper_Wick,Lower_Wick,Range,Is_Green,Is_Hammer
Date,,,,,,,,,,,,,
2025-10-09,1231.069946,1237.500000,1211.819946,1214.250000,2824100,1208.637795,1099.696148,16.819946,6.430054,2.430054,25.680054,True,False
2025-10-10,1220.079956,1247.000000,1219.099976,1229.000000,4279800,1209.851394,1101.251298,8.920044,18.000000,0.979980,27.900024,False,False
2025-10-13,1219.030029,1231.119995,1206.810059,1221.349976,2460800,1211.059995,1102.789198,2.319946,9.770020,12.219971,24.309937,False,False
2025-10-14,1215.349976,1224.290039,1203.099976,1214.750000,2362400,1211.947195,1104.205348,0.599976,8.940063,11.650024,21.190063,True,False
2025-10-15,1203.290039,1219.619995,1201.609985,1213.010010,2025200,1213.055596,1105.601098,9.719971,6.609985,1.680054,18.010010,False,False
2025-10-16,1183.589966,1216.709961,1176.000000,1212.150024,2832600,1213.157795,1106.981298,28.560059,4.559937,7.589966,40.709961,False,False
2025-10-17,1199.359985,1203.119995,1178.949951,1183.599976,2957100,1213.537595,1108.475948,15.760010,3.760010,4.650024,24.170044,True,False
2025-10-20,1238.560059,1247.760010,1206.900024,1207.880005,3986200,1214.075996,1110.212148,30.680054,9.199951,0.979980,40.859985,True,False
2025-10-21,1241.349976,1248.599976,1231.760010,1242.829956,6394800,1214.536794,1111.985248,1.479980,5.770020,9.589966,16.839966,False,False


In [20]:
# Generate Trading Signal
# Signal: Buy when we have a green hammer AND price is above 50 DMA AND 50 DMA is above 200 DMA (bullish trend)
nflx_data['Buy_Signal'] = (
    (nflx_data['Is_Hammer']) &  # Green hammer pattern
    (nflx_data['Close'] > nflx_data['SMA_50']) &  # Price above 50 DMA
    (nflx_data['SMA_50'] > nflx_data['SMA_200'])  # 50 DMA above 200 DMA (golden cross territory)
)

# Calculate returns for signal evaluation
# We'll hold for 20 days (approximately 1 month) after each signal
holding_period = 20
nflx_data['Future_Return'] = (nflx_data['Close'].shift(-holding_period) - nflx_data['Close']) / nflx_data['Close'] * 100

# Calculate daily returns for Sharpe Ratio
nflx_data['Daily_Return'] = nflx_data['Close'].pct_change() * 100

# Get all buy signals
buy_signals = nflx_data[nflx_data['Buy_Signal']].copy()

print(f"Trading Signal Generated:")
print(f"=" * 60)
print(f"Strategy: Buy on Green Hammer + Price > 50 DMA + 50 DMA > 200 DMA")
print(f"Holding Period: {holding_period} days")
print(f"\nTotal Buy Signals: {nflx_data['Buy_Signal'].sum()}")
print(f"\nSample signals:")
buy_signals[['Close', 'SMA_50', 'SMA_200', 'Future_Return']].head(10)

Trading Signal Generated:
Strategy: Buy on Green Hammer + Price > 50 DMA + 50 DMA > 200 DMA
Holding Period: 20 days

Total Buy Signals: 7

Sample signals:


Price,Close,SMA_50,SMA_200,Future_Return
Date,,,,
2021-09-16,586.500000,543.496398,526.221749,8.064789
2021-09-27,592.640015,550.314799,529.126900,13.333551
2023-07-03,441.440002,377.853200,321.487200,-0.638820
2023-07-11,440.209991,389.417400,326.516700,-0.433885
2024-05-28,649.000000,608.434403,505.761051,4.420648
2024-09-11,681.469971,659.351000,596.254951,6.744248
2024-10-24,754.549988,704.296196,633.736050,18.942415


In [21]:
# Calculate Signal Performance Metrics Including Sharpe Ratio
signal_returns = buy_signals['Future_Return'].dropna()

if len(signal_returns) > 0:
    avg_return = signal_returns.mean()
    median_return = signal_returns.median()
    win_rate = (signal_returns > 0).sum() / len(signal_returns) * 100
    profitable_trades = (signal_returns > 0).sum()
    losing_trades = (signal_returns < 0).sum()
    best_trade = signal_returns.max()
    worst_trade = signal_returns.min()
    
    # Calculate Sharpe Ratio for the strategy
    # Sharpe Ratio = (Mean Return - Risk-Free Rate) / Standard Deviation of Returns
    risk_free_rate = 0  # Assuming 0% for simplicity, can adjust to current T-bill rate
    strategy_std = signal_returns.std()
    sharpe_ratio = (avg_return - risk_free_rate) / strategy_std if strategy_std > 0 else 0
    
    # Calculate buy-and-hold Sharpe Ratio for comparison
    buy_hold_returns = nflx_data['Daily_Return'].dropna()
    buy_hold_mean = buy_hold_returns.mean()
    buy_hold_std = buy_hold_returns.std()
    buy_hold_sharpe = (buy_hold_mean - 0) / buy_hold_std if buy_hold_std > 0 else 0
    # Annualize buy-and-hold Sharpe (252 trading days)
    buy_hold_sharpe_annual = buy_hold_sharpe * np.sqrt(252)
    
    # Calculate buy-and-hold return for comparison
    buy_hold_return = (nflx_data['Close'].iloc[-1] - nflx_data['Close'].iloc[0]) / nflx_data['Close'].iloc[0] * 100
    
    print(f"Signal Performance Analysis ({holding_period}-day holding period)")
    print(f"=" * 70)
    print(f"\nSignal Statistics:")
    print(f"  Total Signals Generated: {len(buy_signals)}")
    print(f"  Signals with Complete Data: {len(signal_returns)}")
    print(f"\nReturn Metrics:")
    print(f"  Average Return per Trade: {avg_return:.2f}%")
    print(f"  Median Return per Trade: {median_return:.2f}%")
    print(f"  Best Trade: {best_trade:.2f}%")
    print(f"  Worst Trade: {worst_trade:.2f}%")
    print(f"  Standard Deviation: {strategy_std:.2f}%")
    print(f"\nRisk-Adjusted Performance:")
    print(f"  Sharpe Ratio (Strategy): {sharpe_ratio:.3f}")
    print(f"  Sharpe Ratio (Buy-and-Hold, Annualized): {buy_hold_sharpe_annual:.3f}")
    print(f"\nWin/Loss Statistics:")
    print(f"  Win Rate: {win_rate:.2f}%")
    print(f"  Profitable Trades: {profitable_trades}")
    print(f"  Losing Trades: {losing_trades}")
    print(f"\nBenchmark Comparison:")
    print(f"  Buy-and-Hold Return (5 years): {buy_hold_return:.2f}%")
    print(f"  Strategy Avg Return per Signal: {avg_return:.2f}%")
    
    if avg_return > 0:
        print(f"\n✓ Strategy shows PROFITABLE average return!")
    else:
        print(f"\n✗ Strategy shows negative average return")
else:
    print("No complete signals found with the current criteria.")
    sharpe_ratio = 0
    buy_hold_sharpe_annual = 0

Signal Performance Analysis (20-day holding period)

Signal Statistics:
  Total Signals Generated: 7
  Signals with Complete Data: 7

Return Metrics:
  Average Return per Trade: 7.20%
  Median Return per Trade: 6.74%
  Best Trade: 18.94%
  Worst Trade: -0.64%
  Standard Deviation: 7.12%

Risk-Adjusted Performance:
  Sharpe Ratio (Strategy): 1.012
  Sharpe Ratio (Buy-and-Hold, Annualized): 0.609

Win/Loss Statistics:
  Win Rate: 71.43%
  Profitable Trades: 5
  Losing Trades: 2

Benchmark Comparison:
  Buy-and-Hold Return (5 years): 128.98%
  Strategy Avg Return per Signal: 7.20%

✓ Strategy shows PROFITABLE average return!


In [22]:
# Visualize the Trading Signal on Candlestick Chart
from plotly.subplots import make_subplots

fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=nflx_data.index,
    open=nflx_data['Open'],
    high=nflx_data['High'],
    low=nflx_data['Low'],
    close=nflx_data['Close'],
    name='NFLX'
))

# Add 50-day moving average
fig.add_trace(go.Scatter(
    x=nflx_data.index,
    y=nflx_data['SMA_50'],
    mode='lines',
    name='50 DMA',
    line=dict(color='blue', width=1.5)
))

# Add 200-day moving average
fig.add_trace(go.Scatter(
    x=nflx_data.index,
    y=nflx_data['SMA_200'],
    mode='lines',
    name='200 DMA',
    line=dict(color='orange', width=1.5)
))

# Add buy signals
buy_signal_dates = nflx_data[nflx_data['Buy_Signal']].index
buy_signal_prices = nflx_data[nflx_data['Buy_Signal']]['Low'] * 0.98  # Place markers slightly below the candle

fig.add_trace(go.Scatter(
    x=buy_signal_dates,
    y=buy_signal_prices,
    mode='markers',
    name='Buy Signal',
    marker=dict(
        symbol='triangle-up',
        size=12,
        color='lime',
        line=dict(color='darkgreen', width=2)
    )
))

# Update layout
fig.update_layout(
    title='Netflix (NFLX) - Trading Signals: Green Hammer + 50 DMA > 200 DMA',
    yaxis_title='Stock Price (USD)',
    xaxis_title='Date',
    template='plotly_white',
    xaxis_rangeslider_visible=True,
    height=700,
    width=1400,
    hovermode='x unified'
)

fig.show()

In [23]:
# Detailed view of all buy signals with their outcomes
signal_details = buy_signals[['Close', 'SMA_50', 'SMA_200', 'Future_Return']].copy()
signal_details.columns = ['Entry Price', '50 DMA', '200 DMA', f'{holding_period}-Day Return (%)']
signal_details = signal_details.round(2)

print(f"All Buy Signals and Their Outcomes:")
print(f"=" * 80)
signal_details

All Buy Signals and Their Outcomes:


,Entry Price,50 DMA,200 DMA,20-Day Return (%)
Date,,,,
2021-09-16,586.50,543.50,526.22,8.06
2021-09-27,592.64,550.31,529.13,13.33
2023-07-03,441.44,377.85,321.49,-0.64
2023-07-11,440.21,389.42,326.52,-0.43
2024-05-28,649.00,608.43,505.76,4.42
2024-09-11,681.47,659.35,596.25,6.74
2024-10-24,754.55,704.30,633.74,18.94


In [24]:
# Summary Report - All Key Metrics
import os

# Calculate all metrics
signal_returns = buy_signals['Future_Return'].dropna()
total_signals = len(buy_signals)
signals_with_data = len(signal_returns)

if len(signal_returns) > 0:
    avg_return = signal_returns.mean()
    median_return = signal_returns.median()
    win_rate = (signal_returns > 0).sum() / len(signal_returns) * 100
    profitable_trades = (signal_returns > 0).sum()
    losing_trades = (signal_returns < 0).sum()
    best_trade = signal_returns.max()
    worst_trade = signal_returns.min()
    strategy_std = signal_returns.std()
    
    # Calculate Sharpe Ratios
    risk_free_rate = 0
    sharpe_ratio = (avg_return - risk_free_rate) / strategy_std if strategy_std > 0 else 0
    
    buy_hold_returns = nflx_data['Daily_Return'].dropna()
    buy_hold_mean = buy_hold_returns.mean()
    buy_hold_std = buy_hold_returns.std()
    buy_hold_sharpe = (buy_hold_mean - 0) / buy_hold_std if buy_hold_std > 0 else 0
    buy_hold_sharpe_annual = buy_hold_sharpe * np.sqrt(252)
else:
    avg_return = median_return = win_rate = 0
    profitable_trades = losing_trades = 0
    best_trade = worst_trade = strategy_std = 0
    sharpe_ratio = buy_hold_sharpe_annual = 0

# Buy and hold comparison
buy_hold_return = (nflx_data['Close'].iloc[-1] - nflx_data['Close'].iloc[0]) / nflx_data['Close'].iloc[0] * 100

# Stock price metrics
current_price = nflx_data['Close'].iloc[-1]
start_price = nflx_data['Close'].iloc[0]
five_year_high = nflx_data['High'].max()
five_year_low = nflx_data['Low'].min()
current_50dma = nflx_data['SMA_50'].iloc[-1]
current_200dma = nflx_data['SMA_200'].iloc[-1]

# Date range
start_date_str = nflx_data.index[0].strftime('%Y-%m-%d')
end_date_str = nflx_data.index[-1].strftime('%Y-%m-%d')
total_days = len(nflx_data)

# Create text report
report = f"""
================================================================================
                    NETFLIX (NFLX) TRADING ANALYSIS SUMMARY
================================================================================

DATASET OVERVIEW
--------------------------------------------------------------------------------
Ticker:                  NFLX (Netflix)
Period:                  {start_date_str} to {end_date_str}
Total Trading Days:      {total_days}
Data Points:             {nflx_data.shape[0]} rows x {nflx_data.shape[1]} columns

================================================================================
STOCK PRICE PERFORMANCE
================================================================================
Starting Price:          ${start_price:.2f}
Current Price:           ${current_price:.2f}
5-Year High:             ${five_year_high:.2f}
5-Year Low:              ${five_year_low:.2f}
5-Year Return:           {buy_hold_return:.2f}%
Current 50 DMA:          ${current_50dma:.2f}
Current 200 DMA:         ${current_200dma:.2f}

================================================================================
TRADING SIGNAL STRATEGY
================================================================================

STRATEGY DESCRIPTION:
Our strategy identifies buying opportunities when three conditions align: First,
a green hammer candlestick pattern appears, which signals potential bullish
reversal with strong buying pressure at lower prices. Second, the stock price
must be trading above its 50-day moving average, confirming upward momentum.
Third, the 50-day moving average must be above the 200-day moving average,
indicating we are in a broader bullish trend. When all three conditions are met,
we buy and hold for {holding_period} days.

Signal Components:
  1. Green Hammer Pattern - Bullish reversal candlestick
  2. Price > 50 DMA - Confirms upward momentum
  3. 50 DMA > 200 DMA - Golden cross territory (bullish trend)

Holding Period:          {holding_period} days

================================================================================
SIGNAL PERFORMANCE RESULTS
================================================================================
Total Signals Generated:      {total_signals}
Signals with Complete Data:   {signals_with_data}
Average Return per Trade:     {avg_return:.2f}%
Median Return per Trade:      {median_return:.2f}%
Standard Deviation:           {strategy_std:.2f}%
Win Rate:                     {win_rate:.2f}%
Profitable Trades:            {profitable_trades}
Losing Trades:                {losing_trades}
Best Trade:                   {best_trade:.2f}%
Worst Trade:                  {worst_trade:.2f}%

================================================================================
RISK-ADJUSTED PERFORMANCE (SHARPE RATIO)
================================================================================
Strategy Sharpe Ratio:        {sharpe_ratio:.3f}
Buy-and-Hold Sharpe Ratio:    {buy_hold_sharpe_annual:.3f}

Note: Sharpe Ratio measures risk-adjusted returns. Higher values indicate
better risk-adjusted performance. A Sharpe Ratio above 1.0 is considered good,
above 2.0 is very good, and above 3.0 is excellent.

================================================================================
STRATEGY vs BUY-AND-HOLD
================================================================================
Buy-and-Hold (5 years):       {buy_hold_return:.2f}%
Signal Strategy (Avg/trade):  {avg_return:.2f}%
Signal Annualized (approx):   {(avg_return * 252 / holding_period):.2f}%

================================================================================
CONCLUSION
================================================================================
"""

if avg_return > 0:
    report += f"PROFITABLE SIGNAL: The strategy generated an average return of {avg_return:.2f}%\n"
    report += f"per {holding_period}-day trade with a {win_rate:.2f}% win rate.\n\n"
    if win_rate >= 60:
        report += f"The strategy shows strong performance with over 60% of trades being profitable.\n"
    else:
        report += f"The strategy shows moderate performance with mixed results across trades.\n"
    
    if sharpe_ratio > 1.0:
        report += f"\nThe Sharpe Ratio of {sharpe_ratio:.3f} indicates good risk-adjusted returns.\n"
    elif sharpe_ratio > 0:
        report += f"\nThe Sharpe Ratio of {sharpe_ratio:.3f} indicates positive but modest risk-adjusted returns.\n"
else:
    report += f"UNPROFITABLE SIGNAL: The strategy generated negative returns on average.\n"
    report += f"Consider adjusting parameters or trying different indicators.\n"

report += "================================================================================\n"

# Print to console
print(report)

# Save to file in the same directory as the notebook
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
output_file = os.path.join(notebook_dir, 'netflix_analysis_summary.txt')
with open(output_file, 'w') as f:
    f.write(report)

print(f"\nSummary saved to: {output_file}")


                    NETFLIX (NFLX) TRADING ANALYSIS SUMMARY

DATASET OVERVIEW
--------------------------------------------------------------------------------
Ticker:                  NFLX (Netflix)
Period:                  2020-10-23 to 2025-10-22
Total Trading Days:      1255
Data Points:             1255 rows x 16 columns

STOCK PRICE PERFORMANCE
Starting Price:          $488.28
Current Price:           $1118.07
5-Year High:             $1341.15
5-Year Low:              $162.71
5-Year Return:           128.98%
Current 50 DMA:          $1212.39
Current 200 DMA:         $1113.17

TRADING SIGNAL STRATEGY

STRATEGY DESCRIPTION:
Our strategy identifies buying opportunities when three conditions align: First,
a green hammer candlestick pattern appears, which signals potential bullish
reversal with strong buying pressure at lower prices. Second, the stock price
must be trading above its 50-day moving average, confirming upward momentum.
Third, the 50-day moving average must be above the 2